# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

From the problem formulation, we can infer the goal of the project is to classify the students into two groups: those who graduate, and those who fail to graduate. The idea is to know beforehand which students are likely to fail so an early intervention is done. Therefore it is a binary classification problem, with the labels "graduate" set to 1, if graduate, or 0, if fail.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [61]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [62]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.drop('passed', axis=1).shape[1]

# TODO: Calculate passing students
n_passed = sum(student_data['passed'] == 'yes')

# TODO: Calculate failing students
n_failed = sum(student_data['passed'] == 'no')

# TODO: Calculate graduation rate
grad_rate = 100.0 * n_passed / n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [63]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [64]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [65]:
# TODO: Import any additional functionality you may need here

# set seed to be replicable
np.random.seed(42)

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
# I will do this the old fashioned way instead of using train_test_split
shuffled = np.random.permutation(X_all.index)
train_idx = shuffled[:num_train]
test_idx = shuffled[num_train:]

X_train = X_all.iloc[train_idx]
X_test = X_all.iloc[test_idx]
y_train = y_all.iloc[train_idx]
y_test = y_all.iloc[test_idx]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

We have done almost no data analysis up to this point to do any reasonable inference about the most appropriate model. I feel like there is so much more we need to analyze: what is the distribution like for each feature? how is the data sparsity? are there many missing values? how many of the features are continuous and discrete? What is the correlation between the features? These are all questions that could help pick the most appropriate model. 

I also don't know what is coming ahead: are we selecting features somehow? performing some kind of data transformation, like transforming log-normal distributions? using some method of feature extraction such as PCA or ICA? how are we handling missing values? how are we handling outliers? These are also questions that will influence the choice of algorithm.

Honestly, this seems like a "let's throw it all in a box and see what comes out" approach, which I do not agree with. With that said, I will just move on to answer with the little information we've got until now, since this project follows a rigid structure.

**Decision Tree**

The "black and basic" of the classifiers. It has the advantage of having a nice understandable output, which can be printed to an image in scikit-learn, showing the decision criteria used by decision tree at every node and every level of the tree.

The disadvantage is that it is not the best performing model. It would be more effective to use a method uses bagging or boosting of several smaller tress, such as Random Forest and Adaboost.

What makes it a good candidate is specially what I don't know of the data. Since I don't know much up to this point, using a decision tree is a good way to understand my dataset and its features better, since it has an easy to read output and it outputs the feature importances, which is a score that shows how much relevant each feature is to the splits calculated by the tree. 

Decision Trees are widely used, specially as a first algorithm to a data science pipeline, so any example would fit here. In this example [here](https://gbr.pepperdine.edu/2010/08/how-gerber-used-a-decision-tree-in-strategic-decision-making/), it was used by a baby products company to decide "whether to continue using the plastic known as poly-vinyl chloride or, more commonly, PVC". As mentioned, since it has an easy to read output (meaning we can put it on a slide), it is favored in the corporate environment. 

**kNN**

kNN is a non-parametric model, and hence it inherits the weakness and strenghts of non-parametric classifiers. It does not learn the parameter, rather use the full data to classify. The main weakness is that requires more memory space (N) and search time is long. The strength is that it does not require learning, new observations could be used for model without requiring to re-train the classifier.

There are only about 400 data points, with 48 features, most of them binary, so the memory space requirement is not significant, which makes kNN a good candidate. Another point is that students are being added continuously so it would an advantage to have a model that does not require re-training.

kNN is widely used in the finance market for stock price prediction. Here is an [example](http://www.traderplanet.com/articles/view/61262-predicting-sp-futures-with-the-k-nearest-neighbor-algorithm/). It is also discussed in Udacity's/Georgia Tech course Machine Learning for Trading by Professor Tucker.

**Gradient Boosting**

Gradient boosting is all the hype these days, it is even more popular than Random Forests, so I thought I would give it a try here. Boosting is great way to reduce the chance of overfitting without impacting the model performance.

Boosting works by combining the output of multiple weak learners. A weak learner is defined to be any model that can predict the label better than random chance. There are several variations of boosting, but generally it involves taking a subset of the dataset (subset of samples and/or of features), training a weak learner, select another subset of the data which were badly classified, training another weak learner, and iterate away.

There are several boosting algorithms. Gradient boosting works by building the model "in a stage-wise fashion like other boosting methods do, with several weak learners, and it generalizes them by allowing optimization of an arbitrary differentiable loss function." It is "based on the observation by Leo Breiman (which published the first paper on boosting) that boosting can be interpreted as an optimization algorithm on a suitable cost function". [wikipedia](https://en.wikipedia.org/wiki/Gradient_boosting).
 
The difference between AdaBoost and Gradient Boosting is on how to identify shortcomings (subset of the data which were badly classified). AdaBoost identifies it by high-weight data points, while in Gradient Boosting shortcoming are identified by gradients. [cross validated](http://stats.stackexchange.com/questions/164233/intuitive-explanations-of-differences-between-gradient-boosting-trees-gbm-ad)

An advantage is good performance without overfitting. A disadvantage is having to train several weak learners instead of one strong learner, so it takes a long time to learn compared to other single learner algorithms. Several examples of using Gradient Boosting can be found in Kaggle. Here is an [example](https://www.google.com.br/search?q=gradient+boosting+real+examples&oq=gradient+boosting+real+examples&aqs=chrome..69i57.4982j0j4&sourceid=chrome&ie=UTF-8#q=gradient+boosting+kaggle). 

Gradient Boosting is a good choice because it is a high performing ensemble model, and it will give us a baseline to assess the remaining classifiers.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [66]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)
    return (end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)

    return (end-start), f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    training_time = train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    _, train_f1 = predict_labels(clf, X_train, y_train)
    predicting_time, test_f1 = predict_labels(clf, X_test, y_test)
        
    print "F1 score for training set: {:.4f}.".format(train_f1)
    print "F1 score for test set: {:.4f}.".format(test_f1)
    
    return training_time, predicting_time, train_f1, test_f1

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [67]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize the three models
# random already seet with np seed
clf_A = DecisionTreeClassifier()
clf_B = KNeighborsClassifier()
clf_C = GradientBoostingClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
results = []
for clf in [clf_A, clf_B, clf_C]:
    for x, y in [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]:
        results += train_predict(clf, x, y, X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0002 seconds.
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
F1 score for test set: 0.6774.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0012 seconds
Made predictions in 0.0002 seconds.
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
F1 score for test set: 0.6723.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0025 seconds
Made predictions in 0.0003 seconds.
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
F1 score for test set: 0.6500.
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0007 seconds
Made predictions in 0.0012 seconds.
Made predictions in 0.0013 seconds.
F1 score for training set: 0.8201.
F1 score for test set: 0.7519.
Training a KNeighborsClassifier us

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

In [68]:
text = """
** Classifier 1 - Decision Tree **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |{:.4f}|{:.4f}|{:.4f}|{:.4f}
| 200               |{:.4f}|{:.4f}|{:.4f}|{:.4f}
| 300               |{:.4f}|{:.4f}|{:.4f}|{:.4f}

** Classifier 2 - k-Nearest Neighbors **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | {:.4f}|{:.4f}|{:.4f}|{:.4f}
| 200               | {:.4f}|{:.4f}|{:.4f}|{:.4f}
| 300               | {:.4f}|{:.4f}|{:.4f}|{:.4f}

** Classifier 3 - Gradient Boosting **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |{:.4f}|{:.4f}|{:.4f}|{:.4f}
| 200               |{:.4f}|{:.4f}|{:.4f}|{:.4f}
| 300               |{:.4f}|{:.4f}|{:.4f}|{:.4f}
"""

In [69]:
%%capture
print(text.format(*results))

** Classifier 1 - Decision Tree **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0008|0.0003|1.0000|0.6780
| 200               |0.0012|0.0001|1.0000|0.6207
| 300               |0.0018|0.0002|1.0000|0.7097

** Classifier 2 - k-Nearest Neighbors **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0010|0.0016|0.8201|0.7519
| 200               | 0.0006|0.0014|0.8327|0.7328
| 300               | 0.0007|0.0021|0.8604|0.7852

** Classifier 3 - Gradient Boosting **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0537|0.0004|1.0000|0.7377
| 200               |0.0648|0.0003|0.9922|0.7200
| 300               |0.0780|0.0004|0.9734|0.7576

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Given the available data is limited, and based on the results, I would chose k-nearest neighbors. A non parametric model will allow you to quickly incorporate the knowledge of a new observation. Training a kNN with 300 samples only takes 0.0007 of a second, which is 100 times faster than training a parametric classifier with similar performance (Gradient Boosting). On the other hand, predicting will take 0.002 seconds, which is about 4 times longer than the prediction time of Gradient Boosting. The trade-off here is in favour of kNN, even more considering new observations are added every day (students graduate), at almost the same ratio you need to do new predictions. 

F1_score of kNN, which is a balanced ration of precision recall, is also above the results for the best performing parametric classifier in the three different training sizes tested. For 300 samples, kNN has a f1 score of 0.7852, compared to 0.7576 of Gradient Boosting

Finally, kNN shows no sign of overfitting with 300 samples, as opposed to the remaining models. Decision Tree has a .29 (1-0.71) difference between f1 score in the training data versus f1 score in the test data, Gradient Boosting a 0.21 (.97-.76) difference, while kNN has a small difference of .8 (.86-.78). The proximity between the f1 score curve for training and test set is a good sign that model has not overfitted.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **
kNN is a non parametric model, meaning instead of calculating a set of weights or parameters that will be used for prediction, kNN uses all available data points for prediction. 

The process is simple. Imagine you are trying to predict whether or not it is going to rain today. You have data for the past 100 days of umidity and temperature, and you decide to plot this data. In the x-axis is the temperature, in the y-axis the umidity, and each day is a dot in the visualization. If it is day that rained, the dot is blue, if it did not rain, the dot is red. 

You know what the temperature and umidity will be tomorrow, but you don't know if it will rain or not. An easy way to find out is to plot your new data point in the visualization, and look at the dots nearby. Are they blue or red? If most of them are blue, you can infer that the new dot is probably also blue, which means it will rain tomorrow. On the other hand, if the nearby dots are mostly red, it means tomorrow it won't probably rain.


kNN is exactly that. With the help of linear algebra, you can calculate the distance between the dots in a n-dimensional space, not limited to 2, and know which dots are near the new dot you just entered (let's say you also collect data of air polllution and hours of sunlight, then you go from a 2 dimensional space to a 4 dimensional space). The k stands for how many neighbors will you analyze. If k=3, for example, you look at the three closest dots to determine whether it will rain or not.
________
*side note: did not discuss weighting, since the expectation is "to describe the major qualities" and to "avoid discussing algorithm implementation"*

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [70]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.cross_validation import StratifiedShuffleSplit

# TODO: Create the parameters list you wish to tune
parameters = {
    'n_neighbors': range(1,20),
    'p': range(1,5),
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'weights': ['uniform', 'distance']
}

# TODO: Initialize the classifier
clf = KNeighborsClassifier()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)[1])
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)[1])

KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=17, p=2,
           weights='uniform')
Made predictions in 0.0049 seconds.
Tuned model has a training F1 score of 0.8150.
Made predictions in 0.0018 seconds.
Tuned model has a testing F1 score of 0.8054.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
The final model f1 score is 0.8150 for training set and 0.8054 for testing set. Those are much better than the untuned model, which had .8604 for training set and .7852 for the testing set. The difference between then is down to .01, which means we are near the optimal point where the the two score curves approximate. Our model generalizes well to unseen dataset, and it is not overfitted.

There is also an improvement from .785 to .805 in the testing set f1 score. A 0.02 improvement is a great achievement at this point. An 0.805 f2 score is a good result, so given no other premises, we can assume our  model is not underfitted.

It is worth discussing some parameters chosen for the final model. It uses the euclidean distance (which is equivalent to the minkowski distance with the power parameter set to 2), and 17 nearest neighbors. The algorithm used to compute the nearest neighbors is known as ball_tree. Leaf_size has not impact on the model performance, only in training time. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.